# **Import modules**

In [13]:
import pandas as pd
import numpy as np
import yfinance as yf
from newsapi import NewsApiClient
from fredapi import Fred
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta
import pytz
import tkinter as tk
from tkinter import messagebox, ttk
from tkcalendar import Calendar

In [14]:
def validate_date(date_obj):
    """
    Validate and convert date object to datetime in US/Eastern timezone.
    """
    try:
        eastern = pytz.timezone('US/Eastern')
        return date_obj.replace(tzinfo=eastern)
    except Exception:
        return None

def validate_interval(interval):
    """
    Validate the interval against supported yfinance intervals.
    """
    valid_intervals = ['1m', '5m', '30m', '1d', '1wk', '1mo']
    return interval if interval in valid_intervals else None

### **Fetch and Clean historical stock data**

In [15]:
def fetch_historical_stock_data(tickers, start_date, end_date, interval='5m'):
    """
    Fetch historical stock data for a list of tickers at specified interval.
    """
    historical_data = []
    required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    
    eastern = pytz.timezone('US/Eastern')
    if isinstance(start_date, datetime):
        start_date = start_date.astimezone(eastern)
    if isinstance(end_date, datetime):
        end_date = end_date.astimezone(eastern)
    
    try:
        df = yf.download(tickers, start=start_date, end=end_date, interval=interval)
        
        if df.empty:
            print("No data returned. Check if the date range includes trading hours or if tickers are valid.")
            return pd.DataFrame()

        if len(tickers) == 1:
            ticker = tickers[0]
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = [col[0] for col in df.columns]
            
            missing_columns = [col for col in required_columns if col not in df.columns]
            if missing_columns:
                print(f"Missing required columns for {ticker}: {missing_columns}")
                return pd.DataFrame()
            
            df = df.reset_index()
            if 'Date' not in df.columns:
                if df.columns[0] in ['Datetime', 'index', 'DateTime']:
                    df = df.rename(columns={df.columns[0]: 'Date'})
                else:
                    print(f"Unexpected index column: {df.columns[0]}")
                    return pd.DataFrame()
            
            df['Ticker'] = ticker
            df['Adj Close'] = df.get('Adj Close', df['Close'])
            df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'Ticker']]
            print(f"Fetched {len(df)} data points for {ticker} at {interval} interval")
            historical_data.append(df)
        else:
            for ticker in tickers:
                if ticker not in df.columns.get_level_values(1):
                    print(f"No data returned for {ticker}")
                    continue
                ticker_df = df.xs(ticker, level=1, axis=1).copy()
                if ticker_df.empty:
                    print(f"No data returned for {ticker}")
                    continue
                
                missing_columns = [col for col in required_columns if col not in ticker_df.columns]
                if missing_columns:
                    print(f"Missing required columns for {ticker}: {missing_columns}")
                    continue
                
                ticker_df = ticker_df.reset_index()
                if 'Date' not in ticker_df.columns:
                    if ticker_df.columns[0] in ['Datetime', 'index', 'DateTime']:
                        ticker_df = ticker_df.rename(columns={ticker_df.columns[0]: 'Date'})
                    else:
                        print(f"Unexpected index column for {ticker}: {ticker_df.columns[0]}")
                        continue
                
                ticker_df['Ticker'] = ticker
                ticker_df['Adj Close'] = ticker_df.get('Adj Close', ticker_df['Close'])
                ticker_df = ticker_df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'Ticker']]
                print(f"Fetched {len(ticker_df)} data points for {ticker} at {interval} interval")
                historical_data.append(ticker_df)
        
        if not historical_data:
            print("No data collected for any tickers. Possible non-trading day or invalid tickers.")
            return pd.DataFrame()

        combined_df = pd.concat(historical_data, ignore_index=True)
        combined_df['Date'] = pd.to_datetime(combined_df['Date'])
        return combined_df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

In [16]:
def clean_stock_data(data):
    """
    Clean stock data by handling missing values and ensuring data quality.
    """
    if data.empty:
        print("No data to clean.")
        return data

    required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'Ticker']
    if not all(col in data.columns for col in required_columns):
        print("Missing required columns.")
        return data

    price_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close']
    data[price_columns] = data[price_columns].ffill()
    data['Volume'] = data['Volume'].fillna(0)
    data = data.dropna()
    data = data.drop_duplicates(subset=['Ticker', 'Date'], keep='last')
    
    for col in price_columns + ['Volume']:
        data[col] = data[col].clip(lower=0)

    data['Price_Change'] = data.groupby('Ticker')['Close'].pct_change()
    data = data[data['Price_Change'].abs() < 0.05]
    data = data.drop(columns=['Price_Change'], errors='ignore')
    
    return data

# **Fetch news headlines**

In [17]:
def fetch_news_data(tickers, api_key, start_date, end_date):
    """
    Fetch news articles including company-specific, global, and political news.
    """
    newsapi = NewsApiClient(api_key=api_key)
    news_data = []
    
    if isinstance(start_date, datetime):
        start_date = start_date.astimezone(pytz.UTC)
    if isinstance(end_date, datetime):
        end_date = end_date.astimezone(pytz.UTC)
    
    for ticker in tickers:
        try:
            company_query = f"{ticker} stock"
            company_articles = newsapi.get_everything(
                q=company_query,
                from_param=start_date.strftime('%Y-%m-%d'),
                to=end_date.strftime('%Y-%m-%d'),
                language='en',
                sort_by='publishedAt'
            )
            for article in company_articles['articles']:
                news_data.append({
                    'Ticker': ticker,
                    'Date': pd.to_datetime(article['publishedAt']),
                    'Title': article['title'],
                    'Description': article['description'] or '',
                    'Source': article['source']['name'],
                    'URL': article['url'],
                    'Category': 'Company'
                })
            
            global_query = f"{ticker} industry OR global economy OR political {ticker}"
            global_articles = newsapi.get_everything(
                q=global_query,
                from_param=start_date.strftime('%Y-%m-%d'),
                to=end_date.strftime('%Y-%m-%d'),
                language='en',
                sort_by='publishedAt'
            )
            for article in global_articles['articles']:
                news_data.append({
                    'Ticker': ticker,
                    'Date': pd.to_datetime(article['publishedAt']),
                    'Title': article['title'],
                    'Description': article['description'] or '',
                    'Source': article['source']['name'],
                    'URL': article['url'],
                    'Category': 'Global/Political'
                })
                
        except Exception as e:
            print(f"Error fetching news for {ticker}: {e}")
    
    news_df = pd.DataFrame(news_data)
    if not news_df.empty:
        print(f"Fetched {len(news_df)} news articles for {tickers}")
        print(f"News columns: {news_df.columns.tolist()}")
    else:
        print(f"No news articles found for {tickers} in the specified date range.")
    return news_df

In [18]:
def fetch_stock_info(tickers):
    """
    Fetch stock-related data (ISIN, history metadata, dividends, splits, actions, capital gains).
    """
    stock_data = []
    
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            
            # ISIN
            isin = pd.DataFrame({'Ticker': [ticker], 'ISIN': [stock.get_isin()]})
            stock_data.append(isin)
            
            # History Metadata
            history_metadata = pd.DataFrame({'Ticker': [ticker], 'History Metadata': [stock.get_history_metadata()]})
            stock_data.append(history_metadata)
            
            # Dividends
            dividends = stock.get_dividends(start="1900-01-01", end=datetime.now().strftime('%Y-%m-%d')).reset_index()
            dividends['Ticker'] = ticker
            stock_data.append(dividends)
            
            # Splits
            splits = stock.get_splits(start="1900-01-01", end=datetime.now().strftime('%Y-%m-%d')).reset_index()
            splits['Ticker'] = ticker
            stock_data.append(splits)
            
            # Actions
            actions = stock.get_actions().reset_index()
            actions['Ticker'] = ticker
            stock_data.append(actions)
            
            # Capital Gains (if available)
            capital_gains = stock.get_capital_gains().reset_index() if stock.get_capital_gains() is not None else pd.DataFrame()
            if not capital_gains.empty:
                capital_gains['Ticker'] = ticker
                stock_data.append(capital_gains)
            
            print(f"Fetched stock info for {ticker}")
        except Exception as e:
            print(f"Error fetching stock info for {ticker}: {e}")
    
    if stock_data:
        combined_stock = pd.concat(stock_data, ignore_index=True)
        combined_stock.to_csv(f"stock_info_{ticker}.csv", index=False)
        print(f"Saved stock info for {ticker} to stock_info_{ticker}.csv")
        print(f"Stock info columns: {combined_stock.columns.tolist()}")
        return combined_stock
    else:
        print("No stock info retrieved.")
        return pd.DataFrame()

In [19]:
def fetch_financial_reports(tickers):
    """
    Fetch financial statements and related data for tickers.
    """
    financial_data = []
    
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            
            # Income Statement
            income_stmt = stock.get_income_stmt().reset_index()
            income_stmt['Ticker'] = ticker
            income_stmt['Statement'] = 'Income Statement'
            financial_data.append(income_stmt)
            income_stmt.to_csv(f"income_statement_{ticker}.csv", index=False)
            print(f"Saved income statement for {ticker} to income_statement_{ticker}.csv")
            
            # Quarterly Income Statement
            quarterly_income = stock.get_quarterly_income_stmt().reset_index()
            quarterly_income['Ticker'] = ticker
            quarterly_income['Statement'] = 'Quarterly Income Statement'
            financial_data.append(quarterly_income)
            quarterly_income.to_csv(f"quarterly_income_statement_{ticker}.csv", index=False)
            print(f"Saved quarterly income statement for {ticker} to quarterly_income_statement_{ticker}.csv")
            
            # TTM Income Statement
            ttm_income = stock.get_ttm_income_stmt().reset_index()
            ttm_income['Ticker'] = ticker
            ttm_income['Statement'] = 'TTM Income Statement'
            financial_data.append(ttm_income)
            ttm_income.to_csv(f"ttm_income_statement_{ticker}.csv", index=False)
            print(f"Saved TTM income statement for {ticker} to ttm_income_statement_{ticker}.csv")
            
            # Balance Sheet
            balance_sheet = stock.get_balance_sheet().reset_index()
            balance_sheet['Ticker'] = ticker
            balance_sheet['Statement'] = 'Balance Sheet'
            financial_data.append(balance_sheet)
            balance_sheet.to_csv(f"balance_sheet_{ticker}.csv", index=False)
            print(f"Saved balance sheet for {ticker} to balance_sheet_{ticker}.csv")
            
            # Cash Flow
            cash_flow = stock.get_cashflow().reset_index()
            cash_flow['Ticker'] = ticker
            cash_flow['Statement'] = 'Cash Flow'
            financial_data.append(cash_flow)
            cash_flow.to_csv(f"cash_flow_{ticker}.csv", index=False)
            print(f"Saved cash flow for {ticker} to cash_flow_{ticker}.csv")
            
            # Quarterly Cash Flow
            quarterly_cashflow = stock.get_quarterly_cashflow().reset_index()
            quarterly_cashflow['Ticker'] = ticker
            quarterly_cashflow['Statement'] = 'Quarterly Cash Flow'
            financial_data.append(quarterly_cashflow)
            quarterly_cashflow.to_csv(f"quarterly_cashflow_{ticker}.csv", index=False)
            print(f"Saved quarterly cash flow for {ticker} to quarterly_cashflow_{ticker}.csv")
            
            # TTM Cash Flow
            ttm_cashflow = stock.get_ttm_cashflow().reset_index()
            ttm_cashflow['Ticker'] = ticker
            ttm_cashflow['Statement'] = 'TTM Cash Flow'
            financial_data.append(ttm_cashflow)
            ttm_cashflow.to_csv(f"ttm_cashflow_{ticker}.csv", index=False)
            print(f"Saved TTM cash flow for {ticker} to ttm_cashflow_{ticker}.csv")
            
            # Earnings
            earnings = stock.get_earnings().reset_index()
            earnings['Ticker'] = ticker
            earnings['Statement'] = 'Earnings'
            financial_data.append(earnings)
            earnings.to_csv(f"earnings_{ticker}.csv", index=False)
            print(f"Saved earnings for {ticker} to earnings_{ticker}.csv")
            
            # Calendar
            calendar = stock.get_calendar().reset_index()
            calendar['Ticker'] = ticker
            calendar['Statement'] = 'Calendar'
            financial_data.append(calendar)
            calendar.to_csv(f"calendar_{ticker}.csv", index=False)
            print(f"Saved calendar for {ticker} to calendar_{ticker}.csv")
            
            # Earnings Dates
            earnings_dates = stock.get_earnings_dates().reset_index()
            earnings_dates['Ticker'] = ticker
            earnings_dates['Statement'] = 'Earnings Dates'
            financial_data.append(earnings_dates)
            earnings_dates.to_csv(f"earnings_dates_{ticker}.csv", index=False)
            print(f"Saved earnings dates for {ticker} to earnings_dates_{ticker}.csv")
            
            # SEC Filings
            sec_filings = stock.get_sec_filings().reset_index()
            sec_filings['Ticker'] = ticker
            sec_filings['Statement'] = 'SEC Filings'
            financial_data.append(sec_filings)
            sec_filings.to_csv(f"sec_filings_{ticker}.csv", index=False)
            print(f"Saved SEC filings for {ticker} to sec_filings_{ticker}.csv")
            
        except Exception as e:
            print(f"Error fetching financial reports for {ticker}: {e}")
    
    if financial_data:
        combined_financials = pd.concat(financial_data, ignore_index=True)
        combined_financials.to_csv(f"financial_reports_{ticker}.csv", index=False)
        print(f"Saved combined financial reports for {ticker} to financial_reports_{ticker}.csv")
        print(f"Financial reports columns: {combined_financials.columns.tolist()}")
        return combined_financials
    else:
        print("No financial data retrieved.")
        return pd.DataFrame()

In [20]:
def fetch_analysis_and_holdings(tickers):
    """
    Fetch analysis and holdings data for tickers.
    """
    analysis_data = []
    
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            
            # Analyst Price Targets
            analyst_price_targets = pd.DataFrame(stock.get_analyst_price_targets()).T.reset_index()
            analyst_price_targets['Ticker'] = ticker
            analysis_data.append(analyst_price_targets)
            analyst_price_targets.to_csv(f"analyst_price_targets_{ticker}.csv", index=False)
            print(f"Saved analyst price targets for {ticker} to analyst_price_targets_{ticker}.csv")
            
            # Earnings Estimate
            earnings_estimate = pd.DataFrame(stock.get_earnings_estimate()).T.reset_index()
            earnings_estimate['Ticker'] = ticker
            analysis_data.append(earnings_estimate)
            earnings_estimate.to_csv(f"earnings_estimate_{ticker}.csv", index=False)
            print(f"Saved earnings estimate for {ticker} to earnings_estimate_{ticker}.csv")
            
            # Revenue Estimate
            revenue_estimate = pd.DataFrame(stock.get_revenue_estimate()).T.reset_index()
            revenue_estimate['Ticker'] = ticker
            analysis_data.append(revenue_estimate)
            revenue_estimate.to_csv(f"revenue_estimate_{ticker}.csv", index=False)
            print(f"Saved revenue estimate for {ticker} to revenue_estimate_{ticker}.csv")
            
            # Earnings History
            earnings_history = pd.DataFrame(stock.get_earnings_history()).reset_index()
            earnings_history['Ticker'] = ticker
            analysis_data.append(earnings_history)
            earnings_history.to_csv(f"earnings_history_{ticker}.csv", index=False)
            print(f"Saved earnings history for {ticker} to earnings_history_{ticker}.csv")
            
            # EPS Trend
            eps_trend = pd.DataFrame(stock.get_eps_trend()).T.reset_index()
            eps_trend['Ticker'] = ticker
            analysis_data.append(eps_trend)
            eps_trend.to_csv(f"eps_trend_{ticker}.csv", index=False)
            print(f"Saved EPS trend for {ticker} to eps_trend_{ticker}.csv")
            
            # EPS Revisions
            eps_revisions = pd.DataFrame(stock.get_eps_revisions()).T.reset_index()
            eps_revisions['Ticker'] = ticker
            analysis_data.append(eps_revisions)
            eps_revisions.to_csv(f"eps_revisions_{ticker}.csv", index=False)
            print(f"Saved EPS revisions for {ticker} to eps_revisions_{ticker}.csv")
            
            # Growth Estimates
            growth_estimates = pd.DataFrame(stock.get_growth_estimates()).T.reset_index()
            growth_estimates['Ticker'] = ticker
            analysis_data.append(growth_estimates)
            growth_estimates.to_csv(f"growth_estimates_{ticker}.csv", index=False)
            print(f"Saved growth estimates for {ticker} to growth_estimates_{ticker}.csv")
            
            # Funds Data
            funds_data = pd.DataFrame(stock.get_funds_data()).T.reset_index()
            funds_data['Ticker'] = ticker
            analysis_data.append(funds_data)
            funds_data.to_csv(f"funds_data_{ticker}.csv", index=False)
            print(f"Saved funds data for {ticker} to funds_data_{ticker}.csv")
            
            # Insider Purchases
            insider_purchases = stock.get_insider_purchases().reset_index()
            insider_purchases['Ticker'] = ticker
            analysis_data.append(insider_purchases)
            insider_purchases.to_csv(f"insider_purchases_{ticker}.csv", index=False)
            print(f"Saved insider purchases for {ticker} to insider_purchases_{ticker}.csv")
            
            # Insider Transactions
            insider_transactions = stock.get_insider_transactions().reset_index()
            insider_transactions['Ticker'] = ticker
            analysis_data.append(insider_transactions)
            insider_transactions.to_csv(f"insider_transactions_{ticker}.csv", index=False)
            print(f"Saved insider transactions for {ticker} to insider_transactions_{ticker}.csv")
            
            # Insider Roster Holders
            insider_roster_holders = stock.get_insider_roster_holders().reset_index()
            insider_roster_holders['Ticker'] = ticker
            analysis_data.append(insider_roster_holders)
            insider_roster_holders.to_csv(f"insider_roster_holders_{ticker}.csv", index=False)
            print(f"Saved insider roster holders for {ticker} to insider_roster_holders_{ticker}.csv")
            
            # Major Holders
            major_holders = stock.get_major_holders().reset_index()
            major_holders['Ticker'] = ticker
            analysis_data.append(major_holders)
            major_holders.to_csv(f"major_holders_{ticker}.csv", index=False)
            print(f"Saved major holders for {ticker} to major_holders_{ticker}.csv")
            
            # Institutional Holders
            institutional_holders = stock.get_institutional_holders().reset_index()
            institutional_holders['Ticker'] = ticker
            analysis_data.append(institutional_holders)
            institutional_holders.to_csv(f"institutional_holders_{ticker}.csv", index=False)
            print(f"Saved institutional holders for {ticker} to institutional_holders_{ticker}.csv")
            
            # Mutual Fund Holders
            mutualfund_holders = stock.get_mutualfund_holders().reset_index()
            mutualfund_holders['Ticker'] = ticker
            analysis_data.append(mutualfund_holders)
            mutualfund_holders.to_csv(f"mutualfund_holders_{ticker}.csv", index=False)
            print(f"Saved mutual fund holders for {ticker} to mutualfund_holders_{ticker}.csv")
            
        except Exception as e:
            print(f"Error fetching analysis and holdings for {ticker}: {e}")
    
    if analysis_data:
        combined_analysis = pd.concat(analysis_data, ignore_index=True)
        combined_analysis.to_csv(f"analysis_and_holdings_{ticker}.csv", index=False)
        print(f"Saved combined analysis and holdings for {ticker} to analysis_and_holdings_{ticker}.csv")
        print(f"Analysis and holdings columns: {combined_analysis.columns.tolist()}")
        return combined_analysis
    else:
        print("No analysis and holdings data retrieved.")
        return pd.DataFrame()

# **Exploratory data analysis**

In [21]:
def perform_eda(df, financial_df, news_df, analysis_df, info_df, ticker='AAPL', interval='5m'):
    """
    Perform EDA including stock data, financial reports, news, analysis, and info.
    """
    ticker_df = df[df['Ticker'] == ticker].copy()
    
    if ticker_df.empty:
        print(f"No stock data available for ticker {ticker}")
        return
    
    print(f"\nSummary Statistics for {ticker} Stock Data:")
    print(ticker_df.describe())    

In [22]:
def create_gui():
    """
    Create a tkinter GUI to fetch all available financial data.
    """
    def open_calendar(is_start_date):
        top = tk.Toplevel(root)
        top.title("Select Date")
        top.geometry("300x300")
        
        cal = Calendar(top, selectmode='day', date_pattern='yyyy-mm-dd')
        cal.pack(pady=10)
        
        def select_date():
            selected_date = cal.get_date()
            if is_start_date:
                start_date_var.set(selected_date)
            else:
                end_date_var.set(selected_date)
            error_label.config(text="", fg="red")
            top.destroy()
        
        tk.Button(top, text="Select", command=select_date).pack(pady=10)

    def submit():
        start_date_str = start_date_var.get()
        end_date_str = end_date_var.get()
        interval = interval_var.get()
        
        error_label.config(text="", fg="red")
        
        try:
            start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
            end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
            start_date = validate_date(start_date)
            end_date = validate_date(end_date)
        except ValueError:
            error_label.config(text="Invalid date format. Use YYYY-MM-DD.", fg="red")
            return
        
        if start_date is None or end_date is None:
            error_label.config(text="Invalid date selection. Please select valid dates.", fg="red")
            return
        if not validate_interval(interval):
            error_label.config("Invalid interval. Choose from dropdown.", fg="red")
            return
        if end_date < start_date:
            error_label.config(text="End date must be on or after start date.", fg="red")
            return
        
        range_limits = {
            '1m': 7, '5m': 60, '30m': 60, '1d': 730, '1wk': 1825, '1mo': 7300}
        
        max_days = range_limits.get(interval, 60)
        days_diff = (end_date - start_date).days
        if days_diff > max_days:
            error_label.config(text=f"{interval} interval limited to {max_days} days.", fg="red")
            return
        
        root.destroy()
        
        tickers = ['AAPL']
        news_api_key = '6b423b01d98e47859e2ecbc296aa9b2b'
        
        print(f"\nFetching stock data for {tickers} from {start_date.date()} to {end_date.date()} at {interval} interval...")
        try:
            historical_data = fetch_historical_stock_data(tickers, start_date, end_date, interval)
            if historical_data.empty:
                raise ValueError("No historical data retrieved")

            cleaned_data = clean_stock_data(historical_data)
            cleaned_data.to_csv("cleaned_stock_data.csv", index=False)
            
            
            print(f"\nFetching financial reports for {tickers}...")
            financial_data = fetch_financial_reports(tickers)
            if not financial_data.empty:
                print("Financial reports saved to respective ticker files")
            
            print(f"\nFetching analysis and holdings for {tickers}...")
            analysis_data = fetch_analysis_and_holdings(tickers)
            if not analysis_data.empty:
                print("Analysis and holdings saved to respective ticker files")
            
            print(f"\nFetching info and fast info for {tickers}...")
            info_data = fetch_info_and_fast_info(tickers)
            if not info_data.empty:
                print("Info and fast info saved to respective ticker files")
            
            print(f"\nFetching news data for {tickers} from {start_date.date()} to {end_date.date()}...")
            news_data = fetch_news_data(tickers, news_api_key, start_date, end_date)
            if not news_data.empty:
                news_data.to_csv("news_data.csv", index=False)
                print("News data saved to news_data.csv")
            
            print("\nPerforming exploratory data analysis...")
            perform_eda(cleaned_data, financial_data, news_data, analysis_data, info_data, ticker='AAPL', interval=interval)

        except Exception as e:
            print(f"Error processing data: {e}")
            tk.Tk().withdraw()
            messagebox.showerror("Error", f"Error processing data: {e}")

    def cancel():
        root.destroy()

    def clear_error_on_interaction(*args):
        error_label.config(text="", fg="red")

    root = tk.Tk()
    root.title("Stock Data Input")
    root.geometry("400x400")
    
    tk.Label(root, text="Start Date:").pack(pady=10)
    start_date_var = tk.StringVar(value=datetime.now().strftime('%Y-%m-%d'))
    start_date_entry = tk.Entry(root, textvariable=start_date_var, state='readonly')
    start_date_entry.pack()
    tk.Button(root, text="Select Start Date", command=lambda: open_calendar(True)).pack(pady=5)
    start_date_var.trace('w', clear_error_on_interaction)
    
    tk.Label(root, text="End Date:").pack(pady=10)
    end_date_var = tk.StringVar(value=datetime.now().strftime('%Y-%m-%d'))
    end_date_entry = tk.Entry(root, textvariable=end_date_var, state='readonly')
    end_date_entry.pack()
    tk.Button(root, text="Select End Date", command=lambda: open_calendar(False)).pack(pady=5)
    end_date_var.trace('w', clear_error_on_interaction)
    
    tk.Label(root, text="Interval:").pack(pady=10)
    interval_var = tk.StringVar(value='5m')
    interval_menu = ttk.OptionMenu(root, interval_var, '5m', '1m', '5m', '30m', '1d', '1wk', '1mo')
    interval_menu.pack()
    interval_var.trace('w', clear_error_on_interaction)
    
    error_label = tk.Label(root, text="", fg="red", wraplength=350)
    error_label.pack(pady=10)
    
    button_frame = tk.Frame(root)
    button_frame.pack(pady=20)
    tk.Button(button_frame, text="Submit", command=submit).pack(side=tk.LEFT, padx=10)
    tk.Button(button_frame, text="Cancel", command=cancel).pack(side=tk.LEFT, padx=10)
    
    root.mainloop()

In [ ]:
# Ensure inline plotting in Jupyter
%matplotlib inline

# Launch GUI
create_gui()


Fetching stock data for ['AAPL'] from 2025-05-01 to 2025-05-02 at 30m interval...
